## Step by step on using FunctionTree as VAMPYR class to Multi Resolution Analysis calculations

## -------------------------------------------------------------------------------------------------------
### Importing libraries that shall be used to run the code
## -------------------------------------------------------------------------------------------------------

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import vampyr3d as vp

## -------------------------------------------------------------------------------------------------------
### Defining the basis the Multi Resolution Analysis Calculations Basis
## -------------------------------------------------------------------------------------------------------

#### Alters the lower bounds to -(2**min_scale) and the upper bounds to +(2**min_scale)

In [ ]:
min_scale = -4

#### Precision of calculations up until defined decimal place

In [ ]:
prec = 1e-4

#### Multiplies de unit lengh and drags the lower bounds to "corner"*"unit length"

In [ ]:
corner = [-1, -1, -1]
boxes = [2, 2, 2]

#### Defines how large the calculation space will be

In [ ]:
world = vp.BoundingBox(min_scale, corner, boxes)

#### Number of basis functions

In [ ]:
order = 5

#### To select desired basis (LegendreBasis or InterpolatingBasis polynomials) for calculations

In [ ]:
basis = vp.InterpolatingBasis(order)

#### Uses defined variables to create the MultiResolutionAnalysis function 

In [ ]:
MRA = vp.MultiResolutionAnalysis(world, basis, 25)

##### Defining a function to be evaluated in terms of its variable(s)

In [ ]:
def f(x):
    return x[0]**2

## -------------------------------------------------------------------------------------------------------
### Creating/calling an operator to be used on the desired calculation
## -------------------------------------------------------------------------------------------------------

#### Creates the Alpert, Beylkin, Gines, Vozovoi derivative operator called "A".

In [ ]:
A = vp.ABGVOperator(MRA, 0.0, 0.0)

#### Creates a derivative operator based on projection onto B-splines, a smoothingderivative operator, called "B". 

In [ ]:
B = vp.BSOperator(MRA, 1)

#### Creates a convolution operator called "H" with the complex Helmholtz Green’s function kernel.

In [ ]:
H = vp.HelmholtzOperator(MRA, 0.0, 0.0)

#### Creates a convolution operator called "I" with a narrow Gaussian kernel, close to Dirac’s deltafunction.

In [ ]:
I = vp.IdentityConvolution(MRA, 0.0)

#### Creates a derivative operator called "PH" based on Pavel Holoborodko’s smoothing derivative.

In [ ]:
PH = vp.PHOperator(MRA, 1)

#### Creates a convolution operator with the Poisson Green’s function kernel called "P" that runs with defined precision.

In [ ]:
P = vp.PoissonOperator(MRA, prec)

## -------------------------------------------------------------------------------------------------------
### FunctionTree in MRA basis: creating functionTree, projecting it and aplying desired operator
## -------------------------------------------------------------------------------------------------------

#### Creates a Function Tree called "f_tree" in MRA basis 

In [ ]:
f_tree = vp.FunctionTree(MRA)

#### Creates a Function Tree called "d_f_tree", that shall alocate "f_tree", in MRA basis 

In [ ]:
d_f_tree = vp.FunctionTree(MRA)

#### Defines Project: (precision defined "prec"; FunctionTree "f_tree"; function that will be inserted in f_tree "f")

In [ ]:
vp.project(prec, f_tree, f)

#### Applies designed operator (A, B, H, I, PH, P) on a FunctionTree; (precision defined "prec"; output FunctionTree "d_f_Tree"; Poisson  operator "P"; FunctionTree in which the operator will be applied "f_Tree")

In [ ]:
vp.apply(prec, d_f_tree, P, f_tree)

## -------------------------------------------------------------------------------------------------------
### Functions from FunctionTree
## -------------------------------------------------------------------------------------------------------

#### .evalf = To evaluate the FunctionTree in a given coordinate, i.e.: [0.0, 0.0, 0.0]

In [ ]:
d_f_tree.evalf([0.0, 0.0, 0.0])

#### .clear = Leaves the tree in the same state as after construction

In [ ]:
d_f_tree.clear()

#### .integrate = Integral of the FunctionTree over all space

In [ ]:
d_f_tree.integrate()

#### .normalize = Rescales the function by its norm, fixed grid

In [ ]:
d_f_tree.normalize()

#### .getEndValues = Returns the ending wavelength values

In [ ]:
d_f_tree.getEndValues([0,0])

#### .setEndValues = Sets ending wavelength values

In [ ]:
d_f_tree.setEndValues([0,0])

#### .saveTree = Write the tree structure to disk, for later use; argument file name will get a ".tree" file extension

In [ ]:
d_f_tree.saveTree(func)

#### .loadTree = Read a previously stored tree structure from disk; argument file name will get a ".tree" file extension

In [ ]:
d_f_tree.loadTree(func)

#### .crop = Reduce the accuracy of the tree by deleting nodes which have a higher precision than the requested precison

In [ ]:
b = 10
d_f_tree.crop(5.0, 1.0, b<10)

#### .multiply = Multiplies the function by a given number and it is multiplied by the function

In [ ]:
d_f_tree.multiply(2, d_f_tree)

#### .add = Multiply the function by a given number and adds it to the function

In [ ]:
d_f_tree.add(2, d_f_tree)

#### .power = Raise an existing function to a given power

In [ ]:
d_f_tree.power(2.0)

#### .square = Multiply an existing function with itself

In [ ]:
d_f_tree.square()

#### .rescale = The leaf node point values of the output function will be in-place multiplied by the given coefficient, no grid refinement

In [ ]:
d_f_tree.rescale(0.9)

#### .getNChunks = Get number of chunks

In [ ]:
d_f_tree.getNChunks()

#### .getNChunksUsed = Write size of tree

In [ ]:
d_f_tree.getNChunksUsed()

#### .printSerialIndices = Print Serial Indices

In [ ]:
d_f_tree.printSerialIndices()